In [1]:
!python --version

Python 3.10.12


In [2]:
import sys

sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR")

In [3]:
!pip install -qq -r /kaggle/input/monodtr-library-ver2/MonoDTR/requirement.txt
!pip install -qq coloredlogs
!pip install -qq ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.4 MB/s eta 0:00:00


In [4]:
sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR/scripts")
sys.path.append("/kaggle/input/pretrain-monodtr-base")
sys.path.append("/kaggle/input/checkpoint")

In [5]:
import os
import sys
import torch

# Add the directory containing the .so file to the system path
sys.path.append('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn')

# Check if the .so file exists
print(os.listdir('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn'))

# Print Python version
print("Python version:", sys.version)

# Set LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn:' + os.environ.get('LD_LIBRARY_PATH', '')

# Attempt to import the module
try:
    import deform_conv_ext
    print("Import successful!")
except ImportError as e:
    print("Import failed:", e)

# If needed, run the setup or make script
# !python setup.py build
# !python setup.py install
# or
# !bash make.sh


['deform_conv.py', 'deform_conv_ext.cpython-310-x86_64-linux-gnu.so', 'make.sh', 'src', '__init__.py', 'setup.py']
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Import successful!


In [6]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Check your settings.")


CUDA is available!
Number of GPUs available: 1
Current GPU: 0
GPU Name: Tesla P100-PCIE-16GB


In [7]:
# Create a large tensor and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(10000, 10000, device=device)  # This should allocate GPU memory
print("Tensor created on GPU.")
print("Allocated GPU Memory:", torch.cuda.memory_allocated(device))
print("Cached GPU Memory:", torch.cuda.memory_reserved(device))


Tensor created on GPU.
Allocated GPU Memory: 400556032
Cached GPU Memory: 400556032


In [8]:
import torch.nn as nn
import torch
import torch
from torch.nn import Module, Dropout
from einops import rearrange
import torch.nn.functional as F
from ptflops import get_model_complexity_info

In [9]:
# -------------------------- RMSNorm --------------------------
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: (B, L, D)
        norm_x = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return self.scale * norm_x

# -------------------------- FLIP Module --------------------------
def flip(x, dim):
    return torch.flip(x, dims=[dim])

In [10]:
# -------------------------- BiMamba2 Block (Conv Fusion) --------------------------
class BiMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u):
        # u: (B, L, D)
        x = self.linear_x(u)          # (B, L, D)
        bf = self.linear_bf(u)        # (B, L, D)
        bb = self.linear_bb(u)        # (B, L, D)
        z = self.linear_z(u)          # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)         # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)         # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)           # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)           # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)     
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)# (B, D, L)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z
        af = self.norm_f(af)                      # (B, L, D)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))  # (B, D, L)
        ab = F.gelu(ab)                                # (B, D, L)

        z_flip = flip(z, dim=1)                        # (B, L, D)
        ab = self.ssms_b(ab).transpose(1, 2)           # (B, L, D)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip   # (B, D, L)
        ab = self.norm_b(ab)                           # (B, L, D)
        ab = flip(ab, dim=1)                           # (B, L, D)

        out = self.output_linear(af + ab)         # (B, L, D)                 # (B, L, D)
        return out

In [11]:
# -------------------------- CrossMamba2 Module --------------------------
class CrossMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u1, u2):
        # u1: context (B, L, D), u2: depth (B, L, D)
        x = self.linear_x(u1)               # (B, L, D)
        z = self.linear_z(u1)               # (B, L, D)
        bf = self.linear_bf(u2)             # (B, L, D)
        bb = self.linear_bb(u2)             # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)   # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)   # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)     # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)     # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z  # af conv with z
        af = self.norm_f(af)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))
        ab = F.gelu(ab).transpose(1, 2)           # (B, L, D)

        z_flip = flip(z, dim=1)
        ab = self.ssms_b(ab.transpose(1, 2)).transpose(1, 2)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip  # (B, D, L)
        ab = self.norm_b(ab)
        ab = flip(ab, dim=1)

        out = self.output_linear(af + ab)         # (B, L, D)
        return out

In [12]:
# -------------------------- FFN + Norm --------------------------
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)

In [13]:
# -------------------------- Depth Aware Mamba (DTR-Compatible Input) --------------------------
class DepthAwareMamba(nn.Module):
    def __init__(self, output_channel_num):
        super().__init__()
        self.output_channel_num = output_channel_num

        self.encoder_bimamba = BiMamba2(self.output_channel_num)
        self.encoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.encoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.encoder_norm2 = nn.LayerNorm(self.output_channel_num)

        self.decoder_bimamba = BiMamba2(self.output_channel_num)
        self.cross_bimamba = CrossMamba2(self.output_channel_num)
        self.decoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.decoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm2 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm3 = nn.LayerNorm(self.output_channel_num)

    def forward(self, depth_feat, context_feat, depth_pos=None):
        depth_feat = depth_feat.contiguous()
        context_feat = context_feat.contiguous()
        if depth_pos is not None:
            depth_pos = depth_pos.contiguous()
            context_feat = context_feat + depth_pos
    
        # Encoder on context_feat
        x = self.encoder_bimamba(context_feat.contiguous())
        x = self.encoder_norm1((x + context_feat).contiguous())
        x_ffn = self.encoder_ffn(x.contiguous())
        x = self.encoder_norm2((x + x_ffn).contiguous())
    
        # Decoder on depth_feat and fused context
        d = self.decoder_bimamba(depth_feat.contiguous())
        d = self.decoder_norm1((d + depth_feat).contiguous())
    
        x = self.cross_bimamba(x.contiguous(), d.contiguous())
        x = self.decoder_norm2(x.contiguous())
        x_ffn = self.decoder_ffn(x.contiguous())
        x = self.decoder_norm3((x + x_ffn).contiguous())
        return x


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define AConv unit (Average and Maximum Convolution)
class AConv(nn.Module):
    def __init__(self, channels):
        super(AConv, self).__init__()
        self.avg_pool = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_avg = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.conv_max = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.fuse = nn.Conv2d(2 * channels, channels, kernel_size=1)  # Fuse the two paths

    def forward(self, x):
        avg_out = self.conv_avg(self.avg_pool(x))
        max_out = self.conv_max(self.max_pool(x))
        cat_out = torch.cat([avg_out, max_out], dim=1)
        return self.fuse(cat_out)

# Define Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, dilation=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.dw = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size,
                            stride=1, padding=padding, groups=in_channels, dilation=dilation)
        self.pw = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.pw(self.dw(x))

# Define Focused Multi-Scale Fusion (FMF) module
class FMF(nn.Module):
    def __init__(self, channels=256):
        super(FMF, self).__init__()
        self.channels = channels
        fused_channels = 4 * channels  # 4C thay vì 3C

        # Initial fusion: AConv cho very_large và large, 1x1 cho mid, upsample cho small
        self.acconv_very_large = AConv(channels)
        self.down_very_large1 = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)  # H/4 → H/8
        self.down_very_large2 = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)  # H/8 → H/16

        self.acconv_large = AConv(channels)
        self.down_large = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)  # H/8 → H/16

        self.conv_mid = nn.Conv2d(channels, channels, kernel_size=1)  # H/16 keep

        self.upsample_small = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)  # H/32 → H/16

        # Detail fusion: 4 parallel DWConv với dilations khác nhau
        self.dw1 = DepthwiseSeparableConv(fused_channels, fused_channels, padding=1, dilation=1)
        self.dw2 = DepthwiseSeparableConv(fused_channels, fused_channels, padding=2, dilation=2)
        self.dw3 = DepthwiseSeparableConv(fused_channels, fused_channels, padding=4, dilation=4)
        self.dw4 = DepthwiseSeparableConv(fused_channels, fused_channels, padding=8, dilation=8)  # Thêm cho scale mới

        self.tconv = nn.ConvTranspose2d(fused_channels, channels, kernel_size=2, stride=2)  # Up to H/8

    def forward(self, features):
        # features: [very_large (H/4), large (H/8), mid (H/16), small (H/32)] all 256ch post-proj
        very_large, large, mid, small = features

        # Initial fusion: align to H/16
        x_very_large = self.acconv_very_large(very_large)
        x_very_large = self.down_very_large1(x_very_large)  # H/4 → H/8
        x_very_large = self.down_very_large2(x_very_large)  # H/8 → H/16

        x_large = self.acconv_large(large)
        x_large = self.down_large(x_large)  # H/8 → H/16

        x_mid = self.conv_mid(mid)  # H/16

        x_small = self.upsample_small(small)  # H/32 → H/16

        # Concat to F (4C, H/16, W/16)
        F = torch.cat([x_very_large, x_large, x_mid, x_small], dim=1)

        # Detail fusion: 4 DW
        dw1_out = self.dw1(F)
        dw2_out = self.dw2(F)
        dw3_out = self.dw3(F)
        dw4_out = self.dw4(F)

        stacked = torch.stack([dw1_out, dw2_out, dw3_out, dw4_out], dim=0)
        reduced = torch.sum(stacked, dim=0)
        f_feature = reduced + F

        f_out = self.tconv(f_feature)  # To H/8, 256ch
        return f_out

In [15]:
from visualDet3D.networks.backbones.dla import dla102
from visualDet3D.networks.backbones.dlaup import DLAUp
from visualDet3D.networks.detectors.dfe import DepthAwareFE
from visualDet3D.networks.detectors.dpe import DepthAwarePosEnc
from visualDet3D.networks.detectors.dtr import DepthAwareTransformer
import math
import time
import csv

In [16]:
import timm

class MonoDTRCore(nn.Module):
    def __init__(self, backbone_arguments=dict()):
        super(MonoDTRCore, self).__init__()

        # Swin-T backbone with features_only=True to ensure feature pyramid output
        self.backbone = timm.create_model('swin_tiny_patch4_window7_224', pretrained='coco',features_only=True, img_size=(288, 1280))
        # self.backbone = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)
        self.first_level = 1  # Kept for reference, but no longer used
        # Feature pyramid channels from Swin-T
        self.channels = [96, 192, 384, 768]
        self.output_channel_num = 256
        self.proj1_4 = nn.Conv2d(self.channels[0], self.output_channel_num, kernel_size=1)
        self.proj1_8 = nn.Conv2d(self.channels[1], self.output_channel_num, kernel_size=1)
        self.proj1_16 = nn.Conv2d(self.channels[2], self.output_channel_num, kernel_size=1)
        self.proj1_32 = nn.Conv2d(self.channels[3], self.output_channel_num, kernel_size=1)
        self.FMF = FMF(channels = self.output_channel_num)
        
        self.dpe = DepthAwarePosEnc(self.output_channel_num)
        self.depth_embed = nn.Embedding(100, self.output_channel_num)
        self.dtr = DepthAwareTransformer(self.output_channel_num)
        self.dfe = DepthAwareFE(self.output_channel_num)
        self.img_conv = nn.Conv2d(self.output_channel_num, self.output_channel_num, kernel_size=3, padding=1)

    def forward(self, x):
        """
        Forward pass of MonoDTRCore.
        
        Input:
            x: dict with key 'image' containing (N, 3, H, W)
        
        Output:
            feat: (N, C, H/8, W/8) - Processed feature map for compatibility with dtr
            depth: (N, D, H, W) - Depth prediction at 1/1 resolution
        """
        if 'image' not in x:
            raise ValueError("Input dictionary must contain 'image' key")
        img = x['image']  # (N, 3, H, W)
        assert img.shape[2] == 288 and img.shape[3] == 1280, f"Expected shape [N, 3, 288, 1280], got {img.shape}"

        if img.dim() != 4:
            raise ValueError(f"Expected 4D input tensor (N, C, H, W), got shape {img.shape}")
        
        if img.shape[1] != 3:
            raise ValueError(
                f"Expected input image to have 3 channels (RGB), got {img.shape[1]} channels. "
                "Please check your data pipeline to ensure the input is correctly formatted as (N, 3, H, W). "
                "If the input has 7 channels (e.g., multi-spectral), preprocess it to 3 channels or modify the backbone."
            )

        # Extract features using standard forward method
        features = self.backbone(img)  # [(N, H/4, W/4, 96), (N, H/8, W/8, 192), (N, H/16, W/16, 384), (N, H/32, W/32, 768)]

        # Permute features from (N, H, W, C) to (N, C, H, W)
        features = [f.permute(0, 3, 1, 2) for f in features]  # [(N, 96, H/4, W/4), (N, 192, H/8, W/8), (N, 384, H/16, W/16), (N, 768, H/32, W/32)]

        feat = [self.proj1_4(features[0]),self.proj1_8(features[1]),self.proj1_16(features[2]),self.proj1_32(features[3])]        
        x = self.FMF(feat)

        # Proceed with downstream processing
        N, C, H, W = x.shape  # H=H/8, W=W/8 (reduced resolution)
        # print(f"X shape: {x.shape}")
        depth, depth_guide, depth_feat = self.dfe(x) # depth: (N, D, H/8, W/8), depth_guide: (N, num_classes, H/8, W/8), depth_feat: (N, 256, H/8, W/8)
        depth_feat = depth_feat.permute(0, 2, 3, 1).view(N, H * W, C)
        depth_guide = depth_guide.argmax(1)
        depth_emb = self.depth_embed(depth_guide).view(N, H * W, C)
        depth_emb = self.dpe(depth_emb, (H, W))
        img_feat = x + self.img_conv(x)
        img_feat = img_feat.permute(0, 2, 3, 1)
        # print(f"img_feat shape: {img_feat.shape}")
        img_feat = img_feat.view(N, H * W, C)
        feat = self.dtr(depth_feat, img_feat, depth_emb)
        feat = feat.permute(0, 2, 1).view(N, C, H, W)

        return feat, depth

In [17]:
from visualDet3D.networks.heads.detection_3d_head import AnchorBasedDetection3DHead
from visualDet3D.networks.heads.depth_losses import bin_depths, DepthFocalLoss
from visualDet3D.networks.utils.registry import DETECTOR_DICT

In [18]:
class MonoDTR(nn.Module):
    def __init__(self, network_cfg):
        super(MonoDTR, self).__init__()

        self.obj_types = network_cfg.obj_types

        self.build_head(network_cfg)

        self.build_core(network_cfg)

        self.network_cfg = network_cfg

    def build_core(self, network_cfg):
        self.mono_core = MonoDTRCore(network_cfg.mono_backbone)

    def build_head(self, network_cfg):
        self.bbox_head = AnchorBasedDetection3DHead(
            **(network_cfg.head)
        )
        self.depth_loss = DepthFocalLoss(96)

    def train_forward(self, left_images, annotations, P2, depth_gt=None):
        
        features, depth = self.mono_core(dict(image=left_images, P2=P2))
        
        depth_output   = depth
        
        features = features.contiguous()
        P2 = P2.contiguous()
        left_images = left_images.contiguous()
        
        try:
            cls_preds, reg_preds = self.bbox_head(
                dict(
                    features=features,
                    P2=P2,
                    image=left_images
                )
            )
        except RuntimeError as e:
            print(f"RuntimeError: {e}")
            raise
            
        anchors = self.bbox_head.get_anchor(left_images, P2)

        cls_loss, reg_loss, loss_dict = self.bbox_head.loss(cls_preds, reg_preds, anchors, annotations, P2)
        
        depth_gt = bin_depths(depth_gt, mode = "LID", depth_min=1, depth_max=80, num_bins=96, target=True)
        
        # if depth_gt is not None: #use in feature size is (H/4, W/4)
        #         depth_gt = F.interpolate(depth_gt.unsqueeze(1), size=depth_output.shape[2:], mode='nearest').squeeze(1)
            
        if reg_loss.mean() > 0 and not depth_gt is None and not depth_output is None:
            
            depth_gt = depth_gt.unsqueeze(1)
            depth_loss = 1.0 * self.depth_loss(depth_output, depth_gt)
            loss_dict['depth_loss'] = depth_loss
            reg_loss += depth_loss

            self.depth_output = depth_output.detach()
        else:
            loss_dict['depth_loss'] = torch.zeros_like(reg_loss)
        return cls_loss, reg_loss, loss_dict

    def test_forward(self, left_images, P2):
        assert left_images.shape[0] == 1 # we recommmend image batch size = 1 for testing

        features, _ = self.mono_core(dict(image=left_images, P2=P2))
        
        cls_preds, reg_preds = self.bbox_head(
                dict(
                    features=features,
                    P2=P2,
                    image=left_images
                )
            )

        anchors = self.bbox_head.get_anchor(left_images, P2)

        scores, bboxes, cls_indexes = self.bbox_head.get_bboxes(cls_preds, reg_preds, anchors, P2, left_images)
        
        return scores, bboxes, cls_indexes

    def forward(self, inputs):

        if isinstance(inputs, list) and len(inputs) >= 3:
            return self.train_forward(*inputs)
        else:
            return self.test_forward(*inputs)

In [19]:
import os
import sys
import numpy as np
from easydict import EasyDict
from tqdm import tqdm
from fire import Fire
import coloredlogs
import logging
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from _path_init import *
from visualDet3D.networks.utils.registry import DETECTOR_DICT, DATASET_DICT, PIPELINE_DICT
from visualDet3D.networks.utils.utils import BackProjection, BBox3dProjector, get_num_parameters
from visualDet3D.evaluator.kitti.evaluate import evaluate
import visualDet3D.data.kitti.dataset
from visualDet3D.utils.timer import Timer
from visualDet3D.utils.utils import LossLogger, cfg_from_file
from visualDet3D.networks.optimizers import optimizers, schedulers

In [20]:
import torch
from torch.cuda.amp import autocast  # For AMP
import torch.optim as optim
from visualDet3D.utils.utils import LossLogger
from visualDet3D.utils.utils import compound_annotation

def My_train_mono_detection(data, module: nn.Module,
                         optimizer: optim.Optimizer,
                         writer: SummaryWriter = None,
                         loss_logger: LossLogger = None,
                         global_step: int = None,
                         epoch_num: int = None,
                         cfg: EasyDict = EasyDict(),
                         iter_num: int = None):
    if iter_num is None:
        iter_num = 0  # Fallback, but manage externally
    steps = cfg.trainer.accumulation_steps
    is_accum_start = (iter_num % steps == 0)
    if is_accum_start and optimizer is not None:
        optimizer.zero_grad(set_to_none=True)  # Faster than zero_grad()

    images, P2, labels, bbox2d, bbox_3d, depth = data
    # Assume batch=1, so lists of len=1

    # Handle empty annotations gracefully
    max_length = np.max([len(label) for label in labels]) if labels else 0
    if max_length == 0:
        return  # Still increments iter_num externally

    annotation = compound_annotation(labels, max_length, bbox2d, bbox_3d, cfg.obj_types)
    annotation_tensor = images.new(annotation).cuda()  # Inherit dtype/device

    # Forward pass with AMP for efficiency
    # with autocast(device_type='cuda', enabled=cfg.get('use_amp', True)):
    cls_loss, reg_loss, loss_dict = module([
        images.cuda().float().contiguous(),
        annotation_tensor,
        P2.cuda(),
        depth.cuda().contiguous()
    ])
    cls_loss = cls_loss.mean()
    reg_loss = reg_loss.mean()
    total_loss = cls_loss + reg_loss
    # loss_dict['total_loss'] = total_loss.item()  # Add for logging

    # Log unscaled micro-batch losses
    if loss_logger is not None:
        loss_logger.update(loss_dict)
    del loss_dict

    if optimizer is not None:
        # Check for invalid losses
        if torch.isnan(total_loss) or torch.isinf(total_loss):
            print(f"Warning: Invalid loss at iter {iter_num}: {total_loss.item()}")
            # Optionally: optimizer.zero_grad(); return
            total_loss = torch.tensor(0.0, device=total_loss.device, requires_grad=True)

        scaled_loss = total_loss / steps
        scaled_loss.backward()

    # Update every accumulation_steps
    if (iter_num + 1) % steps == 0:
        # Gradient clipping
        if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
            torch.nn.utils.clip_grad_norm_(module.parameters(), cfg.optimizer.clipped_gradient_norm)
        else:
            print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)  # Reset after step for next cycle

In [21]:
from collections import OrderedDict  # Add this import
def main(config="/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py", experiment_name="default", world_size=1, local_rank=-1):
    """Main function for the training script.

    KeywordArgs:
        config (str): Path to config file.
        experiment_name (str): Custom name for the experitment, only used in tensorboard.
        world_size (int): Number of total subprocesses in distributed training. 
        local_rank: Rank of the process. Should not be manually assigned. 0-N for ranks in distributed training (only process 0 will print info and perform testing). -1 for single training. 
    """

    ## Get config
    cfg = cfg_from_file(config)

    ## Collect distributed(or not) information
    cfg.dist = EasyDict()
    cfg.dist.world_size = world_size
    cfg.dist.local_rank = local_rank
    is_distributed = local_rank >= 0 # local_rank < 0 -> single training
    is_logging     = local_rank <= 0 # only log and test with main process
    is_evaluating  = local_rank <= 0

    ## Setup writer if local_rank > 0
    recorder_dir = os.path.join(cfg.path.log_path, experiment_name + f"config={config}")
    if is_logging: # writer exists only if not distributed and local rank is smaller
        ## Clean up the dir if it exists before
        if os.path.isdir(recorder_dir):
            os.system("rm -r {}".format(recorder_dir))
            print("clean up the recorder directory of {}".format(recorder_dir))
        writer = SummaryWriter(recorder_dir)

        ## Record config object using pprint
        import pprint

        formatted_cfg = pprint.pformat(cfg)
        writer.add_text("config.py", formatted_cfg.replace(' ', '&nbsp;').replace('\n', '  \n')) # add space for markdown style in tensorboard text
    else:
        writer = None

    ## Set up GPU and distribution process
    if is_distributed:
        cfg.trainer.gpu = local_rank # local_rank will overwrite the GPU in configure file
    gpu = min(cfg.trainer.gpu, torch.cuda.device_count() - 1)
    torch.backends.cudnn.benchmark = getattr(cfg.trainer, 'cudnn', False)
    torch.cuda.set_device(gpu)
    if is_distributed:
        torch.distributed.init_process_group(backend='nccl', init_method='env://')
    print(local_rank)
 
    ## define datasets and dataloader.
    dataset_train = DATASET_DICT[cfg.data.train_dataset](cfg)
    dataset_val = DATASET_DICT[cfg.data.val_dataset](cfg, "validation")

    dataloader_train = DataLoader(dataset_train, num_workers=cfg.data.num_workers,
                                  batch_size=cfg.data.batch_size, collate_fn=dataset_train.collate_fn, shuffle=local_rank<0, drop_last=True,
                                  sampler=torch.utils.data.DistributedSampler(dataset_train, num_replicas=world_size, rank=local_rank, shuffle=True) if local_rank >= 0 else None)

    detector = MonoDTR(cfg.detector)         
            
    ## Convert to cuda
    if is_distributed:
        detector = torch.nn.SyncBatchNorm.convert_sync_batchnorm(detector)
        detector = torch.nn.parallel.DistributedDataParallel(detector.cuda(), device_ids=[gpu], output_device=gpu)
    else:
        detector = detector.cuda()
    
    ## define optimizer and weight decay
    optimizer = optimizers.build_optimizer(cfg.optimizer, detector)

    ## define scheduler
    scheduler_config = getattr(cfg, 'scheduler', None)
    scheduler = schedulers.build_scheduler(scheduler_config, optimizer)
    is_iter_based = getattr(scheduler_config, "is_iter_based", False)

    ## define loss logger
    training_loss_logger =  LossLogger(writer, 'train') if is_logging else None

    ## training pipeline
    if 'training_func' in cfg.trainer:
        # training_dection = PIPELINE_DICT[cfg.trainer.training_func] #hoanbi1
        training_dection = My_train_mono_detection
    else:
        raise KeyError

    ## Get evaluation pipeline
    if 'evaluate_func' in cfg.trainer:
        evaluate_detection = PIPELINE_DICT[cfg.trainer.evaluate_func]
        print("Found evaluate function {}".format(cfg.trainer.evaluate_func))
    else:
        evaluate_detection = None
        print("Evaluate function not found")


    ## timer is used to estimate eta
    timer = Timer()

    print('Num training images: {}'.format(len(dataset_train)))

    resume = True
    epoch = 0
    freeze = True
    Load_lr = True
    if resume == True:
        ckpt_path = os.path.join("/kaggle/input/monodtr-resume-checkpoint", "checkpoint_resume.pt")
        dict_checkpoint = torch.load(ckpt_path, map_location="cpu")
        detector.load_state_dict(dict_checkpoint["state_dict_backbone"])
        l_rate = optimizer.param_groups[0]['lr']
        optimizer.load_state_dict(dict_checkpoint["state_optimizer"])
        
        if Load_lr == False:
            # opt_state = dict_checkpoint["state_optimizer"]

            # # Chỉ load phần state (moments), không load param_groups
            # optimizer_state = {"state": opt_state["state"], "param_groups": optimizer.state_dict()["param_groups"]}
            for param_group in optimizer.param_groups:
                param_group["lr"] = l_rate  
        else:
            scheduler.load_state_dict(dict_checkpoint["state_lr_scheduler"])
            epoch = dict_checkpoint["epoch"]
        
        print(f"Resuming training at epoch {epoch}")
        del dict_checkpoint
        
        if freeze == True:
            # this is for load pretrain checkpoint
            state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_location='cpu')
            
            new_state_dict = OrderedDict()
            loaded_keys = []  # Track which parameters are loaded
            for k, v in state_dict_pretrain.items():
                if k in detector.state_dict() and v.shape == detector.state_dict()[k].shape:
                    new_state_dict[k] = v
                    loaded_keys.append(k)  # Record loaded keys
                    print(f"Parameter {k} is loaded from pretrain")
        
            detector.load_state_dict(new_state_dict, strict=False)
            
            for name, param in detector.named_parameters():
                # print(name)
                if name in loaded_keys:
                    param.requires_grad = False  # Freeze loaded weights
                else:
                    param.requires_grad = True
    else:
        print("Start training at epoch 0")
        if freeze == True:
            state_dict_pretrain = torch.load("/kaggle/input/checkpoint/MonoDTR_99.pth", map_location='cpu')
            detector.load_state_dict(state_dict_pretrain)
            
            # this is for load pretrain checkpoint
            state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_location='cpu')
            
            new_state_dict = OrderedDict()
            loaded_keys = []  # Track which parameters are loaded
            for k, v in state_dict_pretrain.items():
                if k in detector.state_dict() and v.shape == detector.state_dict()[k].shape:
                    new_state_dict[k] = v
                    loaded_keys.append(k)  # Record loaded keys
                    print(f"Parameter {k} is loaded from pretrain")
        
            detector.load_state_dict(new_state_dict, strict=False)
            
            for name, param in detector.named_parameters():
                # print(name)
                if name in loaded_keys:
                    param.requires_grad = False  # Freeze loaded weights
                else:
                    param.requires_grad = True
        else:
            state_dict_pretrain = torch.load("/kaggle/input/checkpoint/MonoDTR_229.pth", map_location='cpu')
            detector.load_state_dict(state_dict_pretrain)


    for name, param in detector.named_parameters():
        if param.requires_grad == True:
            print(f"Parameter {name} is trainable")
            
    ## Record basic information of the model
    if is_logging:
        string1 = detector.__str__().replace(' ', '&nbsp;').replace('\n', '  \n')
        writer.add_text("model structure", string1) # add space for markdown style in tensorboard text
        num_parameters = get_num_parameters(detector)
        print(f'number of trained parameters of the model: {num_parameters}')
        
    detector.train()
    global_step = 0
    l_rate = optimizer.param_groups[0]['lr']
    print(f"LR before start = {l_rate:.6f}")
    for epoch_num in range(epoch, (60+epoch)):
        ## Start training for one epoch
        torch.cuda.empty_cache()
        detector.train()
        if training_loss_logger:
            training_loss_logger.reset()
        for iter_num, data in enumerate(dataloader_train):
            training_dection(data, detector, optimizer, writer, training_loss_logger, global_step, epoch_num, cfg, iter_num)

            global_step += 1
            if (iter_num + 1) % cfg.trainer.accumulation_steps == 0:
                if is_iter_based:
                    scheduler.step()
    
                if is_logging and global_step % cfg.trainer.disp_iter == 0:
                    ## Log loss, print out and write to tensorboard in main process
                    if 'total_loss' not in training_loss_logger.loss_stats:
                        print(f"\nIn epoch {epoch_num}, iteration:{iter_num}, global_step:{global_step}, total_loss not found in logger.")
                    else:
                        log_str = 'Epoch: {} | Iteration: {}  | Running loss: {:1.5f} | eta:{}'.format(
                            epoch_num, iter_num, training_loss_logger.loss_stats['total_loss'].avg,
                            timer.compute_eta(global_step, len(dataloader_train) * cfg.trainer.max_epochs))
                        print(log_str, end='\r')
                        writer.add_text("training_log/train", log_str, global_step)
                        training_loss_logger.log(global_step)
                        
        # Handle partial accumulation at end of epoch
        if (iter_num + 1) % cfg.trainer.accumulation_steps != 0:
            # Gradient clipping if applicable
            if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
                torch.nn.utils.clip_grad_norm_(detector.parameters(), cfg.optimizer.clipped_gradient_norm)
            else:
                print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            
        if not is_iter_based:
            scheduler.step()
            
        l_rate = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch_num+1} — LR = {l_rate:.6f}")
        
        ## save model in main process if needed
        if is_logging:
            torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                cfg.path.checkpoint_path, '{}_latest.pth'.format(
                    cfg.detector.name)
                )
            )
        if is_logging and (epoch_num + 1) % cfg.trainer.save_iter == 0:
            torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                cfg.path.checkpoint_path, '{}_{}.pth'.format(
                    cfg.detector.name,epoch_num)
                )
            )
        checkpoint = {
                "epoch": epoch_num + 1,
                "global_step": global_step,
                "state_dict_backbone": detector.state_dict(),
                "state_optimizer": optimizer.state_dict(),
                "state_lr_scheduler": scheduler.state_dict()
            }
        torch.save(checkpoint, os.path.join(cfg.path.checkpoint_path, f"checkpoint_resume.pt"))
        print(f"Save checkpoint at epoch {epoch_num+1} successfully!")
        ## test model in main process if needed
        if is_evaluating and evaluate_detection is not None and cfg.trainer.test_iter > 0 and (epoch_num + 1) % cfg.trainer.test_iter == 0:
            print("\n/**** start testing after training epoch {} ******/".format(epoch_num))
            evaluate_detection(cfg, detector.module if is_distributed else detector, dataset_val, writer, epoch_num)
            print("/**** finish testing after training epoch {} ******/".format(epoch_num))

        if is_distributed:
            torch.distributed.barrier() # wait untill all finish a epoch

        if is_logging:
            writer.flush()

In [22]:
%%writefile my_config.py

from easydict import EasyDict as edict
import os 
import numpy as np

cfg = edict()
cfg.obj_types = ['Car']
#cfg.obj_types = ['Car', 'Pedestrian', 'Cyclist']

## trainer
trainer = edict(
    gpu = 0,
    max_epochs = 100, 
    disp_iter = 100,
    save_iter = 5,
    test_iter = 10,
    training_func = "train_mono_detection",
    test_func = "test_mono_detection",
    evaluate_func = "evaluate_kitti_obj",
    accumulation_steps = 1,  # hoanbi1
)

cfg.trainer = trainer

## path
path = edict()
path.data_path = "/kaggle/input/kitti-3d-object-detection-dataset/training" # used in visualDet3D/data/.../dataset
path.test_path = "/kaggle/input/kitti-3d-object-detection-dataset/testing" # used in visualDet3D/data/.../dataset
path.visualDet3D_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D" # The path should point to the inner subfolder
path.project_path = "/kaggle/working/" # or other path for pickle files, checkpoints, tensorboard logging and output files.
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)
path.project_path = os.path.join(path.project_path, 'MonoDTR')
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)

path.log_path = os.path.join(path.project_path, "log")
if not os.path.isdir(path.log_path):
    os.mkdir(path.log_path)

path.checkpoint_path = os.path.join(path.project_path, "checkpoint")
if not os.path.isdir(path.checkpoint_path):
    os.mkdir(path.checkpoint_path)

path.preprocessed_path = os.path.join(path.project_path, "output")
if not os.path.isdir(path.preprocessed_path):
    os.mkdir(path.preprocessed_path)

path.train_imdb_path = os.path.join(path.preprocessed_path, "training")
if not os.path.isdir(path.train_imdb_path):
    os.mkdir(path.train_imdb_path)

path.val_imdb_path = os.path.join(path.preprocessed_path, "validation")
if not os.path.isdir(path.val_imdb_path):
    os.mkdir(path.val_imdb_path)

path.test_imdb_path = os.path.join(path.preprocessed_path, "test")
if not os.path.isdir(path.test_imdb_path):
    os.mkdir(path.test_imdb_path)

path.test_result_path = os.path.join(path.test_imdb_path, "data")
if not os.path.isdir(path.test_result_path):
    os.mkdir(path.test_result_path)

cfg.path = path

## optimizer
optimizer = edict(
    type_name = 'adam',
    keywords = edict(
        lr        = 1e-4,
        weight_decay = 0,
    ),
    clipped_gradient_norm = 0.1
)
cfg.optimizer = optimizer
## scheduler
scheduler = edict(
    type_name = 'CosineAnnealingLR',
    keywords = edict(
        T_max     = cfg.trainer.max_epochs,
        eta_min   = 5e-6,
    )
)
cfg.scheduler = scheduler

## data
data = edict(
    batch_size = 8,
    num_workers = 8,
    rgb_shape = (288, 1280, 3),
    train_dataset = "KittiMonoDataset",
    val_dataset   = "KittiMonoDataset",
    test_dataset  = "KittiMonoTestDataset",
    train_split_file = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'train.txt'),
    val_split_file   = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'val.txt'),
)

data.augmentation = edict(
    rgb_mean = np.array([0.485, 0.456, 0.406]),
    rgb_std  = np.array([0.229, 0.224, 0.225]),
    cropSize = (data.rgb_shape[0], data.rgb_shape[1]),
    crop_top = 100,
)
data.train_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='PhotometricDistort', keywords=edict(distort_prob=1.0, contrast_lower=0.5, contrast_upper=1.5, saturation_lower=0.5, saturation_upper=1.5, hue_delta=18.0, brightness_delta=32)),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='RandomMirror', keywords=edict(mirror_prob=0.5)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
data.test_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
cfg.data = data

## networks
detector = edict()
detector.obj_types = cfg.obj_types
detector.name = 'MonoDTR'
detector.mono_backbone=edict(
)
head_loss = edict(
    fg_iou_threshold = 0.5,
    bg_iou_threshold = 0.4,
    L1_regression_alpha = 5 ** 2,
    focal_loss_gamma = 2.0,
    balance_weight   = [20.0],
    #balance_weight   = [20.0, 40, 40],
    regression_weight = [1, 1, 1, 1, 1, 1, 12, 1, 1, 0.5, 0.5, 0.5, 1], #[x, y, w, h, cx, cy, z, sin2a, cos2a, w, h, l]
)
head_test = edict(
    score_thr=0.75,
    cls_agnostic = False,
    nms_iou_thr=0.4,
    post_optimization=False
)

anchors = edict(
        {
            'obj_types': cfg.obj_types,
            'pyramid_levels':[3],
            'strides': [2 ** 3],
            'sizes' : [24],
            'ratios': np.array([0.5, 1, 2.0]),
            'scales': np.array([2 ** (i / 4.0) for i in range(16)]),
        }
    )

head_layer = edict(
    num_features_in=256,
    num_cls_output=len(cfg.obj_types)+1,
    num_reg_output=12,
    cls_feature_size=256,
    reg_feature_size=256,
)
detector.head = edict(
    num_regression_loss_terms=13,
    preprocessed_path=path.preprocessed_path,
    num_classes     = len(cfg.obj_types),
    anchors_cfg     = anchors,
    layer_cfg       = head_layer,
    loss_cfg        = head_loss,
    test_cfg        = head_test
)
detector.anchors = anchors
detector.loss = head_loss
cfg.detector = detector


Writing my_config.py


In [23]:
def run_training():
    # config_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py"  # Path to your config file
    config_path = "/kaggle/working/my_config.py"  # Path to your config file
    experiment_name = "EXP_NAME"  # Use the defined experiment name
    world_size = 1  # For single GPU training
    local_rank = 0  # Local rank set to 0 as per your command
    %env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
    main(config=config_path, experiment_name=experiment_name, world_size=world_size)

In [24]:
run_training()

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
-1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Found evaluate function evaluate_kitti_obj
Num training images: 3712


<ipython-input-21-9eb83c0e39fb>:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dict_checkpoint = torch.load(ckpt_path, map_location="cpu")


...

Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:98.25, 89.44, 79.64
bev  AP:27.27, 20.17, 17.08
3d   AP:19.15, 14.57, 12.14
aos  AP:97.09, 85.97, 75.91
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:98.25, 89.44, 79.64
bev  AP:63.39, 47.58, 41.34
3d   AP:59.00, 42.90, 36.88
aos  AP:97.09, 85.97, 75.91

/**** finish testing after training epoch 59 ******/
Epoch 61 — LR = 0.000036
Save checkpoint at epoch 61 successfully!
Epoch 62 — LR = 0.000035
Save checkpoint at epoch 62 successfully!
Epoch 63 — LR = 0.000034
Save checkpoint at epoch 63 successfully!
Epoch 64 — LR = 0.000032
Save checkpoint at epoch 64 successfully!
Epoch 65 — LR = 0.000031
Save checkpoint at epoch 65 successfully!
Epoch 66 — LR = 0.000030
Save checkpoint at epoch 66 successfully!
Epoch 67 — LR = 0.000028
Save checkpoint at epoch 67 successfully!
Epoch 68 — LR = 0.000027
Save checkpoint at epoch 68 successfully!
Epoch 69 — LR = 0.000026
Save checkpoint at epoch 69 successfully!
Epoch 70 — LR = 0.000025
Save c

100%|██████████| 3769/3769 [05:50<00:00, 10.75it/s]


Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:98.17, 89.59, 79.80
bev  AP:27.74, 21.26, 17.36
3d   AP:19.32, 14.78, 12.25
aos  AP:96.60, 85.63, 75.55
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:98.17, 89.59, 79.80
bev  AP:64.63, 47.91, 41.69
3d   AP:58.81, 43.01, 38.14
aos  AP:96.60, 85.63, 75.55

/**** finish testing after training epoch 69 ******/
Epoch 71 — LR = 0.000023
Save checkpoint at epoch 71 successfully!
Epoch 72 — LR = 0.000022
Save checkpoint at epoch 72 successfully!
Epoch 73 — LR = 0.000021
Save checkpoint at epoch 73 successfully!
Epoch 74 — LR = 0.000020
Save checkpoint at epoch 74 successfully!
Epoch 75 — LR = 0.000019
Save checkpoint at epoch 75 successfully!
Epoch 76 — LR = 0.000018
Save checkpoint at epoch 76 successfully!
Epoch 77 — LR = 0.000017
Save checkpoint at epoch 77 successfully!
Epoch 78 — LR = 0.000016
Save checkpoint at epoch 78 successfully!
Epoch 79 — LR = 0.000015
Save checkpoint at epoch 79 successfully!
Epoch 80 — LR = 0.000014
Save c

100%|██████████| 3769/3769 [05:38<00:00, 11.13it/s]


Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:96.27, 89.51, 77.70
bev  AP:27.77, 21.10, 17.26
3d   AP:19.48, 14.75, 12.21
aos  AP:95.10, 86.01, 74.08
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:96.27, 89.51, 77.70
bev  AP:63.28, 47.54, 41.42
3d   AP:58.67, 42.71, 36.80
aos  AP:95.10, 86.01, 74.08

/**** finish testing after training epoch 79 ******/
